In [1]:
import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt

from models import AnnLassoRegression, LinearModel
import utils

import warnings
warnings.filterwarnings("ignore")

In [2]:
def recoveries_analysis(pred_features, true_features):
    TP = len(set(pred_features) & set(true_features))
    FP = len(set(pred_features) - set(true_features))

    TPR = TP / len(true_features) if len(true_features) > 0 else 0
    FDR = FP / (TP + FP) if (TP + FP) > 0 else 0
    
    exact_recovery = (set(pred_features) == set(true_features))

    return exact_recovery

In [3]:
s_values = np.arange(0, 31)
n = 130
X = pd.read_csv(f"data/X-n{n}-p300.csv")

X = utils.X_to_tensor(X)

lambda_qut = utils.lambda_qut_regression(X, utils.Custom_act_fun())

results_linear_old, results_linear_new = [],[]
results_network_old, results_network_new = [], []

cont_lin_old, cont_lin_new, cont_net_old, cont_net_new = True, True, True, True

for s in s_values:
    rec_linear_old, rec_linear_new = [], []
    rec_network_old, rec_network_new = [], []
    m = 200
    for _ in range(m):
        y, features = utils.generate_linear_data(X, s)

        if cont_lin_old:
            model_linear_old= LinearModel(path_type=0, lambda_qut=lambda_qut)
            model_linear_old.fit(X, y, False)
            rec = recoveries_analysis(model_linear_old.important_features()[1], features)
            rec_linear_old.append(rec)
        else:
            rec_linear_old.append(0.0)

        if cont_net_old:
            model_network_old = AnnLassoRegression(path_type=0, lambda_qut=lambda_qut)
            model_network_old.fit(X, y, False)
            rec = recoveries_analysis(model_network_old.important_features()[1], features)
            rec_network_old.append(rec)
        else:
            rec_network_old.append(0.0)
        
        if cont_lin_new:
            model_linear_new= LinearModel(path_type=1, lambda_qut=lambda_qut)
            model_linear_new.fit(X, y, False)
            rec = recoveries_analysis(model_linear_new.important_features()[1], features)
            rec_linear_new.append(rec)
        else:
            rec_linear_new.append(0.0)

        if cont_net_new:
            model_network_new = AnnLassoRegression(path_type=1, lambda_qut=lambda_qut)
            model_network_new.fit(X, y, False)
            rec = recoveries_analysis(model_network_new.important_features()[1], features)
            rec_network_new.append(rec)
        else:
            rec_network_new.append(0.0)


    mean_linear_old = np.mean(rec_linear_old)
    mean_linear_new = np.mean(rec_linear_new)
    results_linear_old.append(mean_linear_old)
    results_linear_new.append(mean_linear_new)

    cont_lin_old = mean_linear_old!=0.0
    cont_lin_new = mean_linear_new!=0.0

    mean_network_old = np.mean(rec_network_old)
    mean_network_new = np.mean(rec_network_new)
    results_network_old.append(mean_network_old)
    results_network_new.append(mean_network_new)

    cont_net_old = mean_network_old!=0.0
    cont_net_new = mean_network_new!=0.0
    
    print(s, mean_linear_old, mean_linear_new, mean_network_old, mean_network_new)

0 1.0 1.0 1.0 1.0
1 1.0 0.0 1.0 1.0
2 1.0 0.0 1.0 1.0


KeyboardInterrupt: 